In [ ]:
# import libraries
import nltk

In [ ]:
 !pip install pyspellchecker
!pip install swifter
nltk.download('punkt')
nltk.download('punkt_tab')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=a5f219c86b7d68a18d576ab03de0fa3cbe23d067410d297cd630d1af5cf5cee9
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!pip install emoji
!pip uninstall ace_tools -y
!pip install ace_tools --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.8 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
from transformers import BertTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
spell = SpellChecker()
import swifter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import numpy as np
import emoji
from transformers import AutoTokenizer

In [ ]:
!pip install transformers
from transformers import pipeline
from tqdm import tqdm

In [ ]:
file_path = "/content/Final_Sentiment_Results_Costco.csv"
# file_path = "./Final_Sentiment_Results_Costco.csv"

# Load the dataset
dataframe = pd.read_csv(file_path)

# Display basic information
dataframe.head()

,comment_likes,short_code,productName,productLikes,productUrl,comment_body,title,comment_year,comment_month,category,...,vader_compound_score_model2,vader_neg_score_model2,vader_neu_score_model2,vader_pos_score_model2,vader_sentiment_class_model2,sentiment_twitter_roberta,sentiment_score_twitter_roberta,negative_score_twitter_roberta,neutral_score_twitter_roberta,positive_score_twitter_roberta
0,59,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,the picture is beautiful . four stars because ...,"Great picture, slow Ethernet, good wifi",2024,12,Product,...,-0.2706,0.046,0.928,0.026,Negative,Neutral,0.439497,0.184263,0.439497,0.376240
1,3,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,"i recently upgraded to the la old 77 b4 , and ...",Stunning Picture Quality and Performance,2025,1,Product,...,0.9886,0.000,0.811,0.189,Positive,Positive,0.966512,0.003293,0.030196,0.966512
2,12,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,"great picture , but doesn't play well in with ...",Great picture OS app integration so so,2024,12,Product,...,0.4166,0.049,0.884,0.067,Positive,Neutral,0.435863,0.419206,0.435863,0.144931
3,8,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,"costco was great about honoring the warranty ,...",Love it when it's working right,2025,1,Product,...,0.5994,0.029,0.918,0.053,Positive,Negative,0.387129,0.387129,0.379409,0.233462
4,1,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,sweet . great picture and relatively easy to i...,Sweet.,2025,1,Product,...,0.8119,0.000,0.886,0.114,Positive,Positive,0.718953,0.047756,0.233291,0.718953


In [ ]:
# Install required packages
!pip install bert-extractive-summarizer transformers sentencepiece --quiet

import pandas as pd
import re
from summarizer import Summarizer

# Load the BERT-based extractive summarizer
bert_model = Summarizer()

# Clean text to remove non-ASCII characters and normalize whitespace
def clean_text(text):
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'\s+', ' ', text.strip())
    return text

# Summarization function with fallback to original text if summarization fails or returns empty
def generate_extractive_summaries_force(text_series, max_words=300):
    summaries = []
    fixed_errors = 0

    for text in text_series.fillna("").astype(str):
        original_text = text.strip()
        summary = ""

        try:
            text = clean_text(original_text)

            if len(text.split()) > max_words:
                text = ' '.join(text.split()[:max_words])

            summary = bert_model(text)
            summary = clean_text(summary)

        except Exception:
            fixed_errors += 1
            summary = ""

        # Final fallback if summary is blank
        if not summary.strip():
            summary = clean_text(original_text)

        summaries.append(summary)

    return summaries, fixed_errors

# Apply to entire DataFrame
summaries, fixed_errors = generate_extractive_summaries_force(dataframe['comment_body'])
dataframe['comment_summary'] = summaries

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Print diagnostics
print(f"Summarized all rows in dataframe. Fallbacks used due to errors or blank output: {fixed_errors}")

Summarized all rows in dataframe. Fallbacks used due to errors or blank output: 0


In [ ]:
dataframe.to_csv('summarizedResultsLatestLast.csv', index=False)

In [ ]:
from google.colab import files
files.download('summarizedResultsLatestLast.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sample_df = dataframe.sample(n=20, random_state=42)

In [ ]:
sample_df.to_csv('summarizedSampleLast.csv', index=False)

In [ ]:
from google.colab import files
files.download('summarizedSampleLast.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>